In [21]:
import sys
sys.path.append('/root/VFPUMC02')
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# 1 普通的线性回归

## 1.1 推导过程

## 1.2 python手动实现

In [22]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split

# 载入数据
X_raw, y_raw = load_diabetes(return_X_y=True)
print("原始数据 X 的形状:", X_raw.shape)
print("原始数据 y 的形状:", y_raw.shape)

# 切分训练集和测试集
X_train_raw, X_test_raw, y_train, y_test = train_test_split(
    X_raw, y_raw, test_size=0.3, random_state=42
)
print("训练集 X 的形状:", X_train_raw.shape)
print("测试集 X 的形状:", X_test_raw.shape)

原始数据 X 的形状: (442, 10)
原始数据 y 的形状: (442,)
训练集 X 的形状: (309, 10)
测试集 X 的形状: (133, 10)


In [23]:
def add_bias_column(X):
    """
    给特征矩阵 X (形状 n x m) 额外添加一列全 1，返回新矩阵 (形状 n x (m+1))
    """
    n = X.shape[0]
    ones = np.ones((n, 1))
    return np.hstack([X, ones])

# 给训练集和测试集都添加一列全1
X_train = add_bias_column(X_train_raw)
X_test = add_bias_column(X_test_raw)

print("添加偏置后训练集 X 的形状:", X_train.shape)
print("添加偏置后测试集 X 的形状:", X_test.shape)

添加偏置后训练集 X 的形状: (309, 11)
添加偏置后测试集 X 的形状: (133, 11)


In [24]:
config = {
    'lambda':0.4, #正则项系数
    'lr':1e-2,    # 学习率
    'n_iters':10, # 训练轮数
}

weights = np.zeros(X_train.shape[1])
loss_history = []
n, m = X_train.shape[0], X_train.shape[1]
for i in range(config['n_iters']):
    L = 0.5 * np.sum(np.square(X_train.dot(weights) - y_train)) / n + 0.5 * config['lambda'] * np.sum(np.square(weights))
    dL_w = X_train.T.dot(X_train.dot(weights) - y_train) / n + config['lambda'] * weights
    weights = weights - config['lr'] * dL_w
    loss_history.append(L)
    print('*'*8,L,'*'*8)
    print('weights:{}'.format(weights))

******** 14917.194174757282 ********
weights:[ 8.65123217e-03  7.41284414e-04  2.72063286e-02  1.87218484e-02
  9.42016730e-03  7.58949476e-03 -1.57999544e-02  1.86864467e-02
  2.25925882e-02  1.91357072e-02  1.53902913e+00]
******** 14681.71334388507 ********
weights:[ 1.72496672e-02  1.47292730e-03  5.42576246e-02  3.73481779e-02
  1.87773626e-02  1.51310501e-02 -3.15267520e-02  3.72728336e-02
  4.50634466e-02  3.81459291e-02  3.05650910e+00]
******** 14452.7760232736 ********
weights:[ 2.57957640e-02  2.19505598e-03  8.11551413e-02  5.58796485e-02
  2.80721811e-02  2.26250935e-02 -4.71808134e-02  5.57598996e-02
  6.74134862e-02  5.70318401e-02  4.55274162e+00]
******** 14230.200305270268 ********
weights:[ 3.42899765e-02  2.90779607e-03  1.07900118e-01  7.43169138e-02
  3.73052109e-02  3.00720475e-02 -6.27625561e-02  7.41483761e-02
  8.96436084e-02  7.57946003e-02  6.02802417e+00]
******** 14013.809339832502 ********
weights:[ 4.27327531e-02  3.61127146e-03  1.34493780e-01  9.266062

In [25]:
from classfiers.VF_LinearRegression import VF_LinearRegression

In [26]:
vf_linear = VF_LinearRegression(config)

In [27]:
idx_A = list(range(6))
idx_B = list(range(6,11))
XA_train,XB_train = X_train[:,idx_A], X_train[:,idx_B]
XA_test,XB_test = X_test[:,idx_A], X_test[:,idx_B]
# 打印形状
for name,temp in zip(['XA_train','XB_train','XA_test','XB_test'],[XA_train,XB_train,XA_test,XB_test]):
    print(name,temp.shape)

XA_train (309, 6)
XB_train (309, 5)
XA_test (133, 6)
XB_test (133, 5)


In [28]:
vf_linear.fit(XA_train, XB_train, y_train)

{'B': <classfiers.VF_LinearRegression.ClientB object at 0x7f05f6808800>, 'C': <classfiers.VF_LinearRegression.ClientC object at 0x7f05f8c67c20>}
{'A': <classfiers.VF_LinearRegression.ClientA object at 0x7f05f69a4650>, 'C': <classfiers.VF_LinearRegression.ClientC object at 0x7f05f8c67c20>}
{'A': <classfiers.VF_LinearRegression.ClientA object at 0x7f05f69a4650>, 'B': <classfiers.VF_LinearRegression.ClientB object at 0x7f05f6808800>}
******** 14917.194174757282 ********
A weights : [0.00865123 0.00074128 0.02720633 0.01872185 0.00942017 0.00758949]
B weights : [-0.01579995  0.01868645  0.02259259  0.01913571  1.53902913]
******** 14681.71334388507 ********
A weights : [0.01724967 0.00147293 0.05425762 0.03734818 0.01877736 0.01513105]
B weights : [-0.03152675  0.03727283  0.04506345  0.03814593  3.0565091 ]
******** 14452.776023273602 ********
A weights : [0.02579576 0.00219506 0.08115514 0.05587965 0.02807218 0.02262509]
B weights : [-0.04718081  0.0557599   0.06741349  0.05703184  4.552

In [42]:
# 转换为 ndarray
vf_linear_loss_history_array = np.array(vf_linear.loss_history[0])
loss_history_array = np.array(loss_history)
# 打印误差
vf_linear_loss_history_array - loss_history_array

array([ 0.0000000e+00,  0.0000000e+00,  1.8189894e-12,  0.0000000e+00,
        0.0000000e+00, -1.8189894e-12,  0.0000000e+00,  0.0000000e+00,
        0.0000000e+00,  0.0000000e+00])